# The Herald - Parquet

Parquet is the default standard for analytical usecases - it's a binary, column-oriented file format.

Before we were storing data as text in CSV. To read a single column, we would need to read the whole row, and throw away the data we don't need. CSV is also pure text data, so we need to convert the string to a given datatype each time.

Columnar data is easy to only select the data we want - it's stored together. Parquet also stores metadata about the columns in order to help query engines pick out the correct data

![Parquet file format](images/parquet_format.jpeg)

In [1]:
!du -hs ../data/parquet/steam_reviews.parquet

6.1G	../data/parquet/steam_reviews.parquet


In [2]:
import ibis
from ibis import _

In [3]:
p = ibis.read_parquet('../data/parquet/steam_reviews.parquet', table_name='reviews')

In [4]:
p

DatabaseTable: reviews
  recommendationid               int64
  language                       string
  timestamp_created              timestamp('UTC', 6)
  timestamp_updated              timestamp('UTC', 6)
  voted_up                       boolean
  votes_up                       int64
  votes_funny                    int64
  weighted_vote_score            float64
  comment_count                  int64
  steam_purchase                 boolean
  received_for_free              int64
  written_during_early_access    int64
  hidden_in_steam_china          int64
  steam_china_location           string
  author_steamid                 int64
  author_num_games_owned         int64
  author_num_reviews             int64
  author_playtime_forever        int64
  author_playtime_last_two_weeks int64
  author_playtime_at_review      int64
  author_last_played             timestamp('UTC', 6)
  year_created                   int64
  month_created                  int64
  game_id                        string

In [5]:
print(f"{p.count().execute():,}")

127,572,734


In [13]:
%%timeit
p.language.value_counts().order_by(_.language_count.desc()).to_polars()

416 ms ± 6.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [12]:
%%timeit
p.group_by(p.year_created, p.month_created).aggregate(p.voted_up.sum()).to_polars()

314 ms ± 5.57 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
p.group_by(p.voted_up).aggregate(p.votes_funny.sum(), p.votes_up.sum()).execute()

,voted_up,Sum(votes_funny),Sum(votes_up)
0,False,1760953801658,80917810
1,True,6244949726479,199077852
